这是一个定制化的对股票估值的『毛估估』算法，主要是解决个人的两个需求，
1. 对不赢利的年轻公司进行估值；
2. 将不成熟的、成熟的、低增长的不同公司股票话在一起比较（虽然定量的比较一定是不太靠谱的）。

大体的做法是，
1. 基于通常的DCF方法中假设的『快速成长阶段』和『永续增长阶段』两个阶段，增加了一个『不稳定成长阶段』
2. 对『快速成长阶段』中的收益分配做了改变

具体细节可见下文代码及注释。

In [10]:
# 环境和包配置
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.set_option( 'display.precision',2)

In [11]:
# 全局参数
total_growth_years = 10 # 成长阶段总年数 = 不稳定成长阶段.年数 + 快速成长阶段.年数
terminal_years = 10 # 永续增长阶段.年数
terminal_growth = 0.04 # 永续增长阶段.增长率
discount = 0.1 # 折现率

pd.DataFrame([{
    "成长阶段总年数" : total_growth_years,
    "永续增长阶段.年数" : terminal_years,
    "永续增长阶段.增长率" : terminal_growth,
    "折现率" : discount
}])

,成长阶段总年数,永续增长阶段.年数,永续增长阶段.增长率,折现率
0,10,10,0.04,0.1


In [12]:
# 各公司特征数据
# 相较于传统的DCF计算, 在快速成长阶段和永续增长阶段之外, 增加了一个不稳定成长阶段, 主要是为了支持对不赢利或赢利不稳定的"年轻"公司的估值.
# 公平起见, 这个阶段与快速成长阶段共享总年数, 其期末EPS的估计需要根据个人对特定公司的理解来定, 也是这个毛估估方法的核心.
data_202211 = pd.DataFrame([
    {
        "名称": "网易",
        "不稳定成长阶段.年数": 0,         # 成熟公司, 已过不稳定成长阶段
        "不稳定成长阶段.期末EPS": 7.66,   # 近期财报公布的扣非每股收益
        "快速成长阶段.增长率": 16.60,     # 10年扣非每股收益增长率
    },
    {
        "名称": "腾讯",
        "不稳定成长阶段.年数": 0,         # 成熟公司, 已过不稳定成长阶段
        "不稳定成长阶段.期末EPS": 21.86,  # 近期财报公布的扣非每股收益
        "快速成长阶段.增长率": 20.00,     # 10年扣非每股收益增长率
    },
    {
        "名称": "快手",
        "不稳定成长阶段.年数": 5,         # 考虑公司发展情况设置
        "不稳定成长阶段.期末EPS": 12.54,  # 假设20%营收增长，行业20%的净利率, 公司最近12个月的每股收入为 25.20 HKD
        "快速成长阶段.增长率": 16.60,     # 参考行业内成熟企业的数值
    },
    {
        "名称": "京东",
        "不稳定成长阶段.年数": 2,         # 考虑公司发展情况设置
        "不稳定成长阶段.期末EPS": 21.81,  # 近年50%的营收增长, 略低于同行costco的净利率 2.5%, 公司最近12个月的每股收入为 387.78 HKD
        "快速成长阶段.增长率": 16.60,     # 参考行业内成熟企业的数值
    },
    {
        "名称": "拼多多",
        "不稳定成长阶段.年数": 2,         # 考虑公司发展情况设置
        "不稳定成长阶段.期末EPS": 6.75,   # 近年50%的营收增长, 公司最近12个月的扣非每股收益为 3 USD
        "快速成长阶段.增长率": 20.00,     # 参考行业内成熟企业的数值(偏高的)
    },
    {
        "名称": "美团",
        "不稳定成长阶段.年数": 2,         # 考虑公司发展情况设置
        "不稳定成长阶段.期末EPS": 13.11,  # 假设近年30%的营收增长, 20%利润率, 公司最近12个月的每股收入为 38.78 HKD
        "快速成长阶段.增长率": 20.00,     # 参考行业内成熟企业的数值(偏高的)
    },
    {
        "名称": "苹果",
        "不稳定成长阶段.年数": 0,         # 成熟公司, 已过不稳定成长阶段
        "不稳定成长阶段.期末EPS": 6.11,   # 近期财报公布的扣非每股收益
        "快速成长阶段.增长率": 14.90,     # 10年扣非每股收益增长率
    },
    {
        "名称": "微软",
        "不稳定成长阶段.年数": 0,         # 成熟公司, 已过不稳定成长阶段
        "不稳定成长阶段.期末EPS": 9.28,   # 近期财报公布的扣非每股收益
        "快速成长阶段.增长率": 16.80,     # 10年扣非每股收益增长率
    },
    {
        "名称": "BRK",
        "不稳定成长阶段.年数": 0,         # 成熟公司, 已过不稳定成长阶段
        "不稳定成长阶段.期末EPS": 26668,  # 近期财报公布的扣非每股收益
        "快速成长阶段.增长率": 17.10,     # 10年扣非每股收益增长率
    },
])
data_202211

,名称,不稳定成长阶段.年数,不稳定成长阶段.期末EPS,快速成长阶段.增长率
0,网易,0,7.66,16.6
1,腾讯,0,21.86,20.0
2,快手,5,12.54,16.6
3,京东,2,21.81,16.6
4,拼多多,2,6.75,20.0
5,美团,2,13.11,20.0
6,苹果,0,6.11,14.9
7,微软,0,9.28,16.8
8,BRK,0,26668.00,17.1


In [13]:
# 估值计算过程, 针对传统 DCF 方法有一些变化, 但是总体而言并不关键, 关键的还是前面所说对于"不稳定成长阶段.期末EPS"的估计
# 1. 估计 "快速成长阶段.期末EPS", 作为永续阶段计算的起点
unstable_years = data_202211["不稳定成长阶段.年数"]
unstable_eps = data_202211["不稳定成长阶段.期末EPS"]
growth_years = total_growth_years - unstable_years
growth_growth = data_202211["快速成长阶段.增长率"]/100
growth_eps = unstable_eps * (1+growth_growth)**growth_years

# 2. 计算内在价值. 
# 与传统 DCF 方法不同, 从利润留存/分红的角度来考虑, 在这个阶段为了保证快速成长, 绝大部分收益都应投入了扩大生产,
# 所以并不计入快速成长阶段中的各年收益. 同时, 这样一来成长型行业与低增长型在逻辑上更有可比性(虽然一般不会这么干, 纯是心理作用).
# 总的来说, 这个调整并不关键, 但因为不计成长阶段收益, 因此最终估值会相对其他一些估值方法偏小.
q = (1+terminal_growth)/(1+discount) # 中间变量
fair_values = growth_eps*q*(1-q**terminal_years) / ((1-q)*(1+discount)**total_growth_years)

names = data_202211["名称"]
cols = ["不稳定成长阶段.年数", "不稳定成长阶段.期末EPS", "快速成长阶段.年数", "快速成长阶段.增长率", "快速成长阶段.期末EPS", "内在价值"]
output = pd.DataFrame(np.vstack((unstable_years, unstable_eps, growth_years, growth_growth, growth_eps, fair_values)).T, index= names, columns=cols)
output



,不稳定成长阶段.年数,不稳定成长阶段.期末EPS,快速成长阶段.年数,快速成长阶段.增长率,快速成长阶段.期末EPS,内在价值
名称,,,,,,
网易,0.0,7.66,10.0,0.17,35.58,102.08
腾讯,0.0,21.86,10.0,0.20,135.35,388.31
快手,5.0,12.54,5.0,0.17,27.03,77.54
京东,2.0,21.81,8.0,0.17,74.52,213.78
拼多多,2.0,6.75,8.0,0.20,29.02,83.27
美团,2.0,13.11,8.0,0.20,56.37,161.72
苹果,0.0,6.11,10.0,0.15,24.50,70.30
微软,0.0,9.28,10.0,0.17,43.85,125.80
BRK,0.0,26668.00,10.0,0.17,129288.35,370917.47


In [14]:
# 代入现价, 比较安全边际
prices_20230216 = np.array([138.2, 382.6, 61.9, 212.6, 94.46, 148.3, 155.33, 269.32, 469045])
margin_of_safety = (fair_values - prices_20230216) / fair_values
output["现价.20230216"] = prices_20230216
output["安全边际%"] = np.array(margin_of_safety.T * 100)
output

,不稳定成长阶段.年数,不稳定成长阶段.期末EPS,快速成长阶段.年数,快速成长阶段.增长率,快速成长阶段.期末EPS,内在价值,现价.20230216,安全边际%
名称,,,,,,,,
网易,0.0,7.66,10.0,0.17,35.58,102.08,138.20,-35.39
腾讯,0.0,21.86,10.0,0.20,135.35,388.31,382.60,1.47
快手,5.0,12.54,5.0,0.17,27.03,77.54,61.90,20.17
京东,2.0,21.81,8.0,0.17,74.52,213.78,212.60,0.55
拼多多,2.0,6.75,8.0,0.20,29.02,83.27,94.46,-13.44
美团,2.0,13.11,8.0,0.20,56.37,161.72,148.30,8.30
苹果,0.0,6.11,10.0,0.15,24.50,70.30,155.33,-120.95
微软,0.0,9.28,10.0,0.17,43.85,125.80,269.32,-114.08
BRK,0.0,26668.00,10.0,0.17,129288.35,370917.47,469045.00,-26.46
